<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/Tensorflow_Seq2Seq_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
num_samples = 33000

In [4]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [5]:
def preprocess_sentence(sent):
  sent = unicode_to_ascii(sent.lower())
  sent = re.sub(r"([?.!,¿])", r" \1", sent)
  sent = re.sub(r"([?.!,¿])", r" \1", sent)
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)
  return sent

In [7]:
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [8]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      src_line, tar_line, _ = line.strip().split('\t')
      src_line_input = [w for w in preprocess_sentence(src_line).split()]

      tar_line = preprocess_sentence(tar_line)
      tar_line_input = [w for w in ("<SOS> " + tar_line).split()]
      tar_line_target = [w for w in (tar_line + " <EOS>").split()]

      encoder_input.append(src_line_input)
      decoder_input.append(tar_line_input)
      decoder_target.append(tar_line_target)

      if i==num_samples - 1:
        break
  return encoder_input, decoder_input, decoder_target

In [13]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
[['<SOS>', 'va', '!'], ['<SOS>', 'marche', '.'], ['<SOS>', 'bouge', '!'], ['<SOS>', 'salut', '!'], ['<SOS>', 'salut', '.']]
[['va', '!', '<EOS>'], ['marche', '.', '<EOS>'], ['bouge', '!', '<EOS>'], ['salut', '!', '<EOS>'], ['salut', '.', '<EOS>']]


In [14]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [15]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [16]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4606, 프랑스어 단어 집합의 크기 : 8107


In [18]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용
tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

In [20]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 5577 21179 22000 ...  4508 11406 28974]


In [21]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [24]:
print(encoder_input[30997])
print(decoder_input[30997])
print(decoder_target[30997])

[714  26 471  23   0   0   0   0]
[  2 502  33 570  10   0   0   0   0   0   0   0   0   0   0   0]
[502  33 570  10   3   0   0   0   0   0   0   0   0   0   0   0]


In [25]:
n_of_val = int(33000 * 0.1)
print(n_of_val)

3300


In [28]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [29]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


In [30]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [31]:
latent_dim = 50

In [33]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [34]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [35]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [36]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     230300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     405350      input_2[0][0]                    
______________________________________________________________________________________________

In [38]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 26s 54ms/step - loss: 3.1231 - acc: 0.6183 - val_loss: 1.8472 - val_acc: 0.7033
Epoch 2/50
233/233 [==============================] - 9s 40ms/step - loss: 1.6731 - acc: 0.7388 - val_loss: 1.5634 - val_acc: 0.7580
Epoch 3/50
233/233 [==============================] - 9s 41ms/step - loss: 1.4902 - acc: 0.7577 - val_loss: 1.4416 - val_acc: 0.7644
Epoch 4/50
233/233 [==============================] - 9s 40ms/step - loss: 1.3858 - acc: 0.7741 - val_loss: 1.3556 - val_acc: 0.7864
Epoch 5/50
233/233 [==============================] - 10s 41ms/step - loss: 1.2999 - acc: 0.7908 - val_loss: 1.2903 - val_acc: 0.7973
Epoch 6/50
233/233 [==============================] - 10s 41ms/step - loss: 1.2335 - acc: 0.8004 - val_loss: 1.2390 - val_acc: 0.8044
Epoch 7/50
233/233 [==============================] - 10s 41ms/step - loss: 1.1816 - acc: 0.8078 - val_loss: 1.1960 - val_acc: 0.8096
Epoch 8/50
233/233 [==============================] - 10s 41ms/st

In [39]:
encoder_model = Model(encoder_inputs, encoder_states)

In [40]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2,
                                                    initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

In [41]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [42]:
def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,1))
  target_seq[0,0] = tar_to_index['<SOS>']

  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
    output_tokens, h, c=decoder_model.predict([target_seq] + states_value)

    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    decoded_sentence += ' ' + sampled_char

    if (sampled_char=='<EOS>' or len(decoded_sentence) > 50):
      stop_condition = True

    target_seq = np.zeros((1,1))
    target_seq[0,0] = sampled_token_index

    states_value = [h, c]

  return decoded_sentence

In [44]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<SOS>']) and i!=tar_to_index['<EOS>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [45]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index+1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  the fight is over . 
번역문 : le combat est termine . 
예측문 :  le prix est pret . 


원문 :  what do you advise ? 
번역문 : que conseilles tu ? 
예측문 :  que se passe t il il mange ? 


원문 :  please stop . 
번역문 : s il vous plait arretez . 
예측문 :  s il vous plait . 


원문 :  fear nothing . 
번역문 : n aie aucune crainte . 
예측문 :  n as pas peur . 


원문 :  is that you ? 
번역문 : est ce toi ? 
예측문 :  est ce que tu ? 




In [46]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  he came downstairs . 
번역문 : il est descendu . 
예측문 :  il est en train de faire de l aide . 


원문 :  how about tomorrow ? 
번역문 : que dites vous de demain ? 
예측문 :  que se sont nous ? 


원문 :  i have a new car . 
번역문 : j ai une voiture neuve . 
예측문 :  j ai une voiture ! 


원문 :  i need sugar . 
번역문 : j ai besoin de sucre . 
예측문 :  j ai besoin de la tete . 


원문 :  leave tomorrow . 
번역문 : partez demain . 
예측문 :  laisse demain . 


